In [4]:
import os

import xarray as xr
import cfgrib

xr.backends.list_engines()

{'scipy': <ScipyBackendEntrypoint>
   Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html,
 'cfgrib': <CfGribBackend>
   Open GRIB files (.grib, .grib2, .grb and .grb2) in Xarray
   Learn more at https://github.com/ecmwf/cfgrib,
 'rasterio': <RasterioBackend>,
 'store': <StoreBackendEntrypoint>
   Open AbstractDataStore instances in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}

In [13]:
import cfgrib

hrrr_obs_file = '/Users/limingzhou/zhoul/work/energy/grib2/hrrrdata/hrrrsub_2020_01_05_05F0.grib2'

utah_file = '/Users/limingzhou/zhoul/work/energy/grib2/utah/20200105.hrrr.t15z.wrfsfcf00.grib2'

data_file = hrrr_obs_file

backend_kwargs={'filter_by_keys': {'typeOfLevel': 'heightAboveGround', 'level': 10}}
ds_10m = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)

backend_kwargs['filter_by_keys']['level'] = 2
ds_2m = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)

backend_kwargs['filter_by_keys'] = {'stepType': 'instant',
                                    'typeOfLevel': 'surface'}
ds_surface = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)


In [7]:
ds_10m

<xarray.Dataset> Size: 53MB
Dimensions:            (y: 1059, x: 1799)
Coordinates:
    time               datetime64[ns] 8B ...
    step               timedelta64[ns] 8B ...
    heightAboveGround  float64 8B ...
    latitude           (y, x) float64 15MB ...
    longitude          (y, x) float64 15MB ...
    valid_time         datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Data variables:
    u10                (y, x) float32 8MB ...
    v10                (y, x) float32 8MB ...
    si10               (y, x) float32 8MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-07T08:38 GRIB to CDM+CF via cfgrib-0.9.1...

In [11]:
ds_2m

<xarray.Dataset> Size: 61MB
Dimensions:            (y: 1059, x: 1799)
Coordinates:
    time               datetime64[ns] 8B ...
    step               timedelta64[ns] 8B ...
    heightAboveGround  float64 8B ...
    latitude           (y, x) float64 15MB ...
    longitude          (y, x) float64 15MB ...
    valid_time         datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Data variables:
    t2m                (y, x) float32 8MB ...
    sh2                (y, x) float32 8MB ...
    d2m                (y, x) float32 8MB ...
    r2                 (y, x) float32 8MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-07T08:40 GRIB to CDM+CF via cfgrib-0.9.1...

In [14]:
ds_surface

<xarray.Dataset> Size: 107MB
Dimensions:     (y: 1059, x: 1799)
Coordinates:
    time        datetime64[ns] 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    latitude    (y, x) float64 15MB ...
    longitude   (y, x) float64 15MB ...
    valid_time  datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Data variables:
    vis         (y, x) float32 8MB ...
    gust        (y, x) float32 8MB ...
    sp          (y, x) float32 8MB ...
    t           (y, x) float32 8MB ...
    snowc       (y, x) float32 8MB ...
    sde         (y, x) float32 8MB ...
    dswrf       (y, x) float32 8MB ...
    vbdsf       (y, x) float32 8MB ...
    vddsf       (y, x) float32 8MB ...
    blh         (y, x) float32 8MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-07T08:40 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:


def grb_to_grid(grb_obj):
    """Takes a single grb object containing multiple
    levels. Assumes same time, pressure levels. Compiles to a cube"""
    n_levels = len(grb_obj)
    levels = np.array([grb_element['level'] for grb_element in grb_obj])
    indexes = np.argsort(levels)[::-1] # highest pressure first
    cube = np.zeros([n_levels, grb_obj[0].values.shape[0], grb_obj[1].values.shape[1]])
    for i in range(n_levels):
        cube[i,:,:] = grb_obj[indexes[i]].values
    cube_dict = {'data' : cube, 'units' : grb_obj[0]['units'],
                 'levels' : levels[indexes]}
    return cube_dict
        
    
     


### Get all grib files from a folder

In [1]:
g_file = pygrib.open(os.path.join(hrrr_obs_path, 'hrrrsub_2020_01_01_00F0.grib2'))

NameError: name 'pygrib' is not defined

In [ ]:


grb = g_file.select(name='Cloud mixing ratio')
grb_cube=grb_to_grid(grb)

### Get the mina and max datetime

### Get the missing hours

### Extract a rectangle, based on the four corners